# Tokenization and Embeddings

Before feeding text into a Transformer model, it must be converted into a numerical format that the model can understand. This process involves two key steps: **Tokenization** and **Embeddings**.

### What is Tokenization?

Tokenization is the process of splitting text into smaller units called tokens. Tokens can be words, subwords, or characters. Different tokenization methods are used depending on the model architecture and language.

### Types of Tokenization:

1. **Word Tokenization**: Splits text into individual words (e.g., "The quick brown fox").
2. **Subword Tokenization**: Splits words into smaller subword units (e.g., "Transformer" -> "Trans", "##former").
3. **Character Tokenization**: Splits text into individual characters (e.g., "hello" -> "h", "e", "l", "l", "o").

### What are Embeddings?

Embeddings are dense vector representations of tokens. They map tokens to continuous vector spaces, capturing semantic meaning and relationships. 

### Types of Embeddings:

1. **Static Embeddings**: Pre-trained on large corpora and remain fixed during training (e.g., Word2Vec, GloVe).
2. **Contextual Embeddings**: Generated dynamically by Transformer models based on the context of the surrounding words (e.g., BERT, GPT).

Let's explore these concepts in detail and understand how they are applied in practice using the Huggingface `transformers` library.


## Tokenization with Huggingface Transformers

The `transformers` library provides a wide range of pre-trained tokenizers that are optimized for different Transformer models. These tokenizers convert raw text into token IDs that can be fed into a model.

### Example: Tokenizing Text using a Pre-trained Tokenizer
Let's use the BERT tokenizer to tokenize some example text.


In [ ]:
from transformers import AutoTokenizer

In [ ]:
# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/google--bert-base-uncased")

In [ ]:
# Example text
text = "Transformers are revolutionizing NLP!"

In [ ]:
text_2 = "Simeon Harrison is so competent and handsome, he is a unicorn trainer!"

In [ ]:
# Tokenize the text
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

In [ ]:
tokens_2 = tokenizer.tokenize(text_2)
print("Tokens:", tokens_2)

In [ ]:
text_3 = "Large language models on supercomputers"

In [ ]:
tokens_3 = tokenizer.tokenize(text_3)
print("Tokens:", tokens_3)

In [ ]:
# Let's look at the actual numeric representations of the tokens
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

## Exploring Different Tokenization Methods

Huggingface provides several tokenization methods to handle various languages and tasks. Let's explore how different tokenization methods handle the same input text.


In [ ]:
# Example of different tokenizers
bert_base_uncased = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/google--bert-base-uncased"
gpt2 = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/openai-community--gpt2"
xlm_roberta_base = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/facebookai--xlm-roberta-base"

tokenizers = [bert_base_uncased, gpt2, xlm_roberta_base]

for tok_name in tokenizers:
    print(f"\nUsing {tok_name} tokenizer:")
    tokenizer = AutoTokenizer.from_pretrained(tok_name)
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    print("Tokens:", tokens)
    print("Token IDs:", token_ids)

## Tokenizing a Whole Dataset
To tokenize a whole dataset, we first need to define a tokenize funktion.
We can then use the `map()` method to apply the function to the whole dataset.
But before that, we need to load a dataset. Let's use the emotions dataset from the Hugging Face Hub.

In [ ]:
from datasets import load_dataset

We would like to know which tokenizer is behind the variable tokenizer.

In [ ]:
tokenizer

As you can see, we get a lot of information about the bert-base-uncased tokenizer.
Now, let's load the dataset:

In [ ]:
emotions = load_dataset("/leonardo_scratch/fast/EUHPC_D20_063/huggingface/datasets/dair-ai--emotion")

In [ ]:
emotions

Here, you can see the splits the dataset comes in.

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

To see `tokenize()` in action, we pass two examples from the training set:

In [ ]:
print(tokenize(emotions["train"][:2]))

Here you can see, that a tokenized dataset does not just consist of the token ids, but much more, which the model uses for training.  
Now, let's apply `map()`. `batch_size=None` means the tokenize function is mapped to the whole dataset as one big batch.

In [ ]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

In [ ]:
print(emotions_encoded["train"].column_names)

## Embeddings with Transformers

Once text is tokenized, it is converted into embeddings before being fed into the Transformer model. Transformers then create **contextual embeddings** that capture the meaning of words in context.

### Example: Extracting Embeddings from a Pre-trained Model
Let's use a pre-trained BERT model to extract embeddings for some example text.

In [ ]:
from transformers import AutoModel
import torch

In [ ]:
# Load a pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained("/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/google--bert-base-uncased")
model = AutoModel.from_pretrained("/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/google--bert-base-uncased")

# Tokenize the text and convert to input IDs
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
inputs

In [ ]:
# Get the embeddings (output of the model's hidden states)
with torch.no_grad():
    outputs = model(**inputs)

# Extract the last hidden state
last_hidden_state = outputs.last_hidden_state

**Explanation**  
The `with torch.no_grad():` statement creates a context in which operations on tensors will not track gradients.
This is used during inference (when you are using the model to make predictions rather than training it) to save memory and speed up computation by not storing information needed to compute gradients.

`**inputs:` unpacks the inputs dictionary, passing its contents as keyword arguments to the model. `inputs` usually contains keys like:
- 'input_ids': The tokenized input text.
- 'attention_mask': A mask to differentiate real tokens from padding tokens.
- 'token_type_ids' (optional): Indicates the segments in tasks like question answering.

These inputs are then fed into the model for a forward pass.

`outputs:` is a variable that will store the output of the model's forward pass. The specific structure of outputs depends on the type of model used. For most transformer models, the output will be a named tuple or an OrderedDict containing several attributes.

`outputs.last_hidden_state:` extracts the "last hidden state" from the model's output. The last hidden state is a tensor representing the final layer's hidden states (embeddings) for each token in the input sequence.  
The `shape` of last_hidden_state is usually `(batch_size, sequence_length, hidden_size)`, where:
- batch_size: The number of input sequences in the batch.
- sequence_length: The number of tokens in each input sequence.
- hidden_size: The size of the hidden states (e.g., 768 for BERT base).  
Purpose: The last hidden state is a contextual representation of each token in the input sequence, capturing the meaning of the token in the context of the entire sequence.

Let's print that out:

In [ ]:
print("Last Hidden State Shape:", last_hidden_state.shape)
print("Embeddings for each token:\n", last_hidden_state)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
# New text
text = "The fat ginger cat sat on a mat"
inputs = tokenizer(text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

last_hidden_state = outputs.last_hidden_state

In [ ]:
# Convert the embeddings to numpy array
embeddings = last_hidden_state[0].numpy()

# Get the special tokens' IDs
special_tokens_ids = tokenizer.all_special_ids
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Filter out special tokens and their embeddings
filtered_embeddings = []
filtered_tokens = []

In [ ]:
for i, token_id in enumerate(inputs['input_ids'][0]):
    if token_id not in special_tokens_ids:
        filtered_embeddings.append(embeddings[i])
        filtered_tokens.append(tokens[i])

# Apply PCA to reduce dimensions to 2D
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(filtered_embeddings)

**Explanation**  
The `.numpy()` method converts the PyTorch tensor to a NumPy array.
This step is necessary because PCA in scikit-learn operates on NumPy arrays, not PyTorch tensors.
Note: For this conversion to work, the tensor must be on the CPU. If the tensor is on a GPU, you should move it to the CPU first using .cpu().  
E.g.: `embeddings = last_hidden_state[0].cpu().numpy()`

In [ ]:
# Plot the embeddings
plt.figure(figsize=(8, 6))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c='blue')

# Annotate the points with tokens
for i, token in enumerate(filtered_tokens):
    plt.text(reduced_embeddings[i, 0], reduced_embeddings[i, 1], token, fontsize=9)

plt.title("2D PCA of Token Embeddings")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

## Hands-On Exercise: Working with Tokenization and Embeddings

In this exercise, you will:
1. Choose a different pre-trained model from the Huggingface Hub.
2. Tokenize a new text input.
3. Extract and visualize embeddings for the tokens.

### Instructions:

1. Choose a model (e.g., "distilbert-base-uncased").
2. Tokenize the following text: "Huggingface makes NLP easy and accessible."
3. Extract embeddings using the last hidden state of the model.

Try it out below!

In [ ]:
# Your code here:


### (Possible) Solution:

In [ ]:
# Step 1: Choose a model
model_dir = "/leonardo_scratch/fast/EUHPC_D20_063/huggingface/models/distilbert--distilbert-base-uncased"  # Change this to any model of your choice
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModel.from_pretrained(model_dir)

In [ ]:
# Step 2: Tokenize the new text
new_text = "Huggingface makes NLP easy and accessible."
inputs = tokenizer(new_text, return_tensors="pt")

In [ ]:
# Step 3: Extract embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract the last hidden state
new_last_hidden_state = outputs.last_hidden_state

## Conclusion

In this notebook, we explored the concepts of tokenization and embeddings, which are crucial for preparing text input for Transformer models. We demonstrated how to tokenize text using different methods and extract contextual embeddings from pre-trained models.

In [ ]:
# Shut down the kernel to release memory
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(restart=False)